In [1]:
%pip install -U "autogen-agentchat" "autogen-ext[openai]" "python-dotenv"

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import ModelFamily
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

In [3]:
load_dotenv()
model_client = OpenAIChatCompletionClient(
    base_url=os.getenv('BASE_URL'),
    model=os.getenv("MODEL"),
    api_key=os.getenv('API_KEY'),
    model_info={
        "vision":False,
        "json_output":False,
        "function_calling":True,
        "family": ModelFamily.UNKNOWN,
        "structured_output":False
    }
)

In [4]:
def increment_number(number: int) -> int:
    """将一个数字加1。"""
    return number + 1

In [5]:
looped_assistant = AssistantAgent(
    name="looped_assistant",
    model_client=model_client,
    reflect_on_tool_use=True,
    tools=[increment_number],
    system_message="你是一个有用的AI助手，使用工具来增加数字。"
)
text_termination=TextMessageTermination("looped_assistant")

In [6]:
team = RoundRobinGroupChat([looped_assistant], termination_condition=text_termination)

In [7]:
async for message in team.run_stream(task="将数字5递增到10。"): 
    print(type(message).__name__, message)

await model_client.close()

TextMessage source='user' models_usage=None metadata={} content='将数字5递增到10。' type='TextMessage'
ToolCallRequestEvent source='looped_assistant' models_usage=RequestUsage(prompt_tokens=135, completion_tokens=96) metadata={} content=[FunctionCall(id='call_0_a84bd61c-e985-413c-ba91-a5cf7183e252', arguments='{"number": 5}', name='increment_number'), FunctionCall(id='call_1_9028fdc7-6723-48cc-823a-ecc595bdbf17', arguments='{"number": 6}', name='increment_number'), FunctionCall(id='call_2_59fd5dc4-6258-4b2b-b7ef-f643df20206b', arguments='{"number": 7}', name='increment_number'), FunctionCall(id='call_3_c12c4d83-8739-41c3-befb-207ede1b07df', arguments='{"number": 8}', name='increment_number'), FunctionCall(id='call_4_5677db08-42cc-47f7-9d74-f66f52e8db3e', arguments='{"number": 9}', name='increment_number')] type='ToolCallRequestEvent'
ToolCallExecutionEvent source='looped_assistant' models_usage=None metadata={} content=[FunctionExecutionResult(content='6', name='increment_number', call_id='ca